In [1]:
sc

In [17]:
data =  sc.textFile('coalesce_joined_weather_collisions_sorted_fixed.csv')
df = spark.read.load('coalesce_joined_weather_collisions_sorted_fixed.csv', format='csv', header=True, inferSchema=False)


df.registerTempTable('temp')

In [45]:
# df = df.filter(df.LATITUDE != '0')

# test = df.filter(df.DATETIME == '2016-06-20 21:06:00')
# test = df.filter(if '2016-06-20 21' in df.DATETIME)

# test.count()
df.first()

# a.coalesce(1).write.csv('geodata', header=True)

Row(BOROUGH=u'QUEENS', CONTRIBUTING_FACTOR_VEHICLE_1=u'Unspecified', CONTRIBUTING_FACTOR_VEHICLE_2=u'Unspecified', CONTRIBUTING_FACTOR_VEHICLE_3=None, CONTRIBUTING_FACTOR_VEHICLE_4=None, CONTRIBUTING_FACTOR_VEHICLE_5=None, CROSS_STREET_NAME=u'70 DRIVE', DATETIME=u'2016-06-20 21:06:00', LATITUDE=None, LOCATION=None, LONGITUDE=None, NUMBER_OF_CYCLIST_INJURED=u'0', NUMBER_OF_CYCLIST_KILLED=u'0', NUMBER_OF_MOTORIST_INJURED=u'0', NUMBER_OF_MOTORIST_KILLED=u'0', NUMBER_OF_PEDESTRIANS_INJURED=u'0', NUMBER_OF_PEDESTRIANS_KILLED=u'0', NUMBER_OF_PERSONS_INJURED=u'0', NUMBER_OF_PERSONS_KILLED=u'0', OFF_STREET_NAME=None, ON_STREET_NAME=u'METROPOLITAN AVENUE', UNIQUE_KEY=u'3465402', VEHICLE_TYPE_CODE_1=u'PASSENGER VEHICLE', VEHICLE_TYPE_CODE_2=u'PASSENGER VEHICLE', VEHICLE_TYPE_CODE_3=None, VEHICLE_TYPE_CODE_4=None, VEHICLE_TYPE_CODE_5=None, ZIP_CODE=u'11375', _c28=u'58671', Conditions=u'Clear', DateUTC=u'2016-06-21 01:51:00', DatetimeEDT=u'2016-06-20 21:51:00', Dew_PointF=u'63.0', Events=None, Fre

In [51]:
year = "2016%"
query = 'select * from temp where DATETIME LIKE "{0}"'.format(year)
temp = sqlContext.sql(query)
temp.count()

107692

In [37]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def filter_year(x):
    return x.split("-")[0]
    
year_filter = udf(filter_year, StringType())
newcol_df = df.withColumn("YEAR", year_filter("DATETIME"))
unique_years = newcol_df.select("YEAR").distinct()
unique_years_list = unique_years.rdd.flatMap(lambda x: x).collect()

# unique_years = df.select()

In [38]:
unique_years_list

[u'2016', u'2017']

In [43]:
for year in unique_years_list:
    year_wildcard = year+"%"
    query = 'select * from temp where DATETIME LIKE "{0}"'.format(year_wildcard)
    temp = sqlContext.sql(query)
#     print(temp.take(1))
    temp.coalesce(1).write.csv('partitioned_years/joins_in_'+year, header=True)

In [68]:
unique_CONTRIBUTING_FACTOR_1 = newcol_df.select("CONTRIBUTING_FACTOR_VEHICLE_1").distinct()
sorted(unique_CONTRIBUTING_FACTOR_1.rdd.flatMap(lambda x: x).collect())

[None,
 u'Accelerator Defective',
 u'Aggressive Driving/Road Rage',
 u'Alcohol Involvement',
 u'Animals Action',
 u'Backing Unsafely',
 u'Brakes Defective',
 u'Cell Phone (hand-held)',
 u'Cell Phone (hands-free)',
 u'Driver Inattention/Distraction',
 u'Driver Inexperience',
 u'Driverless/Runaway Vehicle',
 u'Drugs (Illegal)',
 u'Failure to Keep Right',
 u'Failure to Yield Right-of-Way',
 u'Fatigued/Drowsy',
 u'Fell Asleep',
 u'Following Too Closely',
 u'Glare',
 u'Headlights Defective',
 u'Illness',
 u'Lane Marking Improper/Inadequate',
 u'Lost Consciousness',
 u'Obstruction/Debris',
 u'Other Electronic Device',
 u'Other Lighting Defects',
 u'Other Vehicular',
 u'Outside Car Distraction',
 u'Oversized Vehicle',
 u'Passenger Distraction',
 u'Passing or Lane Usage Improper',
 u'Pavement Defective',
 u'Pavement Slippery',
 u'Pedestrian/Bicyclist/Other Pedestrian Error/Confusion',
 u'Physical Disability',
 u'Prescription Medication',
 u'Reaction to Other Uninvolved Vehicle',
 u'Shoulders D

In [66]:
unique_CONTRIBUTING_FACTOR_5 = newcol_df.select("CONTRIBUTING_FACTOR_VEHICLE_5").distinct()
sorted(unique_CONTRIBUTING_FACTOR_5.rdd.flatMap(lambda x: x).collect())

[None,
 u'Aggressive Driving/Road Rage',
 u'Alcohol Involvement',
 u'Driver Inattention/Distraction',
 u'Following Too Closely',
 u'Lane Marking Improper/Inadequate',
 u'Other Vehicular',
 u'Oversized Vehicle',
 u'Pavement Slippery',
 u'Reaction to Other Uninvolved Vehicle',
 u'Traffic Control Disregarded',
 u'Unsafe Speed',
 u'Unspecified',
 u'View Obstructed/Limited']

In [69]:
unique_CONTRIBUTING_FACTOR_2 = newcol_df.select("CONTRIBUTING_FACTOR_VEHICLE_2").distinct()
sorted(unique_CONTRIBUTING_FACTOR_2.rdd.flatMap(lambda x: x).collect())

[None,
 u'Accelerator Defective',
 u'Aggressive Driving/Road Rage',
 u'Alcohol Involvement',
 u'Animals Action',
 u'Backing Unsafely',
 u'Brakes Defective',
 u'Cell Phone (hand-held)',
 u'Driver Inattention/Distraction',
 u'Driver Inexperience',
 u'Driverless/Runaway Vehicle',
 u'Drugs (Illegal)',
 u'Failure to Keep Right',
 u'Failure to Yield Right-of-Way',
 u'Fatigued/Drowsy',
 u'Fell Asleep',
 u'Following Too Closely',
 u'Glare',
 u'Illness',
 u'Lane Marking Improper/Inadequate',
 u'Lost Consciousness',
 u'Obstruction/Debris',
 u'Other Electronic Device',
 u'Other Lighting Defects',
 u'Other Vehicular',
 u'Outside Car Distraction',
 u'Oversized Vehicle',
 u'Passenger Distraction',
 u'Passing or Lane Usage Improper',
 u'Pavement Defective',
 u'Pavement Slippery',
 u'Pedestrian/Bicyclist/Other Pedestrian Error/Confusion',
 u'Physical Disability',
 u'Prescription Medication',
 u'Reaction to Other Uninvolved Vehicle',
 u'Steering Failure',
 u'Tire Failure/Inadequate',
 u'Tow Hitch Defec

In [71]:
unique_locations = df.select("LOCATION").distinct()
# sorted(unique_CONTRIBUTING_FACTOR_2.rdd.flatMap(lambda x: x).collect())
unique_locations.count()

57885

In [72]:
locations = df.select("LOCATION")
locations.count()

168340

In [75]:
joins_2015 = spark.read.load('joins_in_2015.csv', format='csv', header=True, inferSchema=False)
joins_2015.cache()

DataFrame[BOROUGH: string, CONTRIBUTING_FACTOR_VEHICLE_1: string, CONTRIBUTING_FACTOR_VEHICLE_2: string, CONTRIBUTING_FACTOR_VEHICLE_3: string, CONTRIBUTING_FACTOR_VEHICLE_4: string, CONTRIBUTING_FACTOR_VEHICLE_5: string, CROSS_STREET_NAME: string, DATETIME: string, LATITUDE: string, LOCATION: string, LONGITUDE: string, NUMBER_OF_CYCLIST_INJURED: string, NUMBER_OF_CYCLIST_KILLED: string, NUMBER_OF_MOTORIST_INJURED: string, NUMBER_OF_MOTORIST_KILLED: string, NUMBER_OF_PEDESTRIANS_INJURED: string, NUMBER_OF_PEDESTRIANS_KILLED: string, NUMBER_OF_PERSONS_INJURED: string, NUMBER_OF_PERSONS_KILLED: string, OFF_STREET_NAME: string, ON_STREET_NAME: string, UNIQUE_KEY: string, VEHICLE_TYPE_CODE_1: string, VEHICLE_TYPE_CODE_2: string, VEHICLE_TYPE_CODE_3: string, VEHICLE_TYPE_CODE_4: string, VEHICLE_TYPE_CODE_5: string, ZIP_CODE: string, _c28: string, Conditions: string, DateUTC: string, DatetimeEDT: string, Dew_PointF: string, Events: string, Freezing: string, Gust_SpeedMPH: string, Humidity: s

In [80]:
unique_locations_2015 = joins_2015.select("LOCATION").distinct()
unique_locations_2015.count()

35408

In [81]:
locations = joins_2015.select("LOCATION")
locations.count()

217417

In [93]:
locations.groupBy("LOCATION").count().orderBy('count', ascending=False).show()

+--------------------+-----+
|            LOCATION|count|
+--------------------+-----+
|                null|34672|
|(40.8100528, -73....|  219|
|(40.6528929, -73....|  136|
|(40.6960346, -73....|  135|
|(40.7606005, -73....|  132|
|(40.7460781, -73....|  116|
|(40.7595273, -73....|  112|
|(40.6757357, -73....|  111|
|(40.7572323, -73....|  108|
|(40.7203211, -73....|  105|
|(40.7602226, -73....|  103|
|(40.6585778, -73....|   96|
|(40.7584292, -73....|   95|
|(40.8571285, -73....|   92|
|(40.8618642, -73....|   90|
|(40.6781627, -73....|   90|
|(40.7832478, -73....|   90|
|(40.8027515, -73....|   88|
|(40.759308, -73.9...|   88|
|(40.7241363, -73....|   87|
+--------------------+-----+
only showing top 20 rows



In [94]:
locations.groupBy("LOCATION").count().orderBy('count', ascending=False).count()

35408

In [95]:
217417-34672

182745

In [ ]:
select("LOCATION").groupBy("LOCATION").count().orderBy('count', ascending=False).show()